In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [2]:
np.random.seed(1)

In [3]:
df = pd.read_csv("peptide.csv")
df.head()

,Patientcode,Age,Sex,Caucasianeth,Smokerever,Smokerpresent,DM,Hypertension,Cerebrovasceventprevious,PAD,...,CKD_EPI,hsCRP,NTProBNP,NTProBNPgreater218,NTproBNP100,MCP1,Gal3,hsTroponinI,TWEAK,SBP
0,497,63,1,1,1,0,0,1,0,0,...,79.744541,1.588,37.3,0,0.373,151.60,9.86,0,123.48,150
1,472,51,0,1,1,0,1,1,0,0,...,65.387946,4.155,556,1,5.56,164.13,18.45,0,238.96,100
2,14,83,1,1,1,0,0,1,0,0,...,69.292452,4.194,1110,1,11.1,155.11,3.79,0.023,618.84,145
3,438,64,1,1,1,0,0,1,0,0,...,89.943755,0.738,35.9,0,0.359,106.86,8.66,0.003,130.13,140
4,477,81,1,1,1,0,1,1,0,0,...,56.371204,0.542,224,1,2.24,209.73,8.23,0.003,116.26,120


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 62 columns):
Patientcode                       699 non-null int64
Age                               699 non-null int64
Sex                               699 non-null int64
Caucasianeth                      699 non-null object
Smokerever                        699 non-null object
Smokerpresent                     699 non-null int64
DM                                699 non-null int64
Hypertension                      699 non-null int64
Cerebrovasceventprevious          699 non-null int64
PAD                               699 non-null int64
HFprevious                        699 non-null int64
EFlower40                         699 non-null int64
AF                                699 non-null int64
TypelastACS                       699 non-null int64
Numbervessels                     699 non-null object
PCIlastevent                      699 non-null int64
DESatlastevent                    699 non-nu

## Data Cleaning

In [5]:
target = 'Death'

### Replace empty or errors by np.nan

In [6]:
# replace empty/erros by np.nan
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace('\t','',regex=True)
df = df.replace(' ','',regex=True)
df = df.replace('\?',np.nan,regex=True)
df = df.replace('\<',np.nan,regex=True)
df = df.replace('#NULL!',np.nan,regex=True)

#df

### Remove outliers

In [7]:
# Drop the column outliers as it is unrelated to the dependent variable
outliers = ['HISTOLOGYCANCER', 'CANCERBEFORE100DAYS','CauseofDeath',\
            'HISTOLOGYCANCERTABULATED','Datestartfollowup','Patientcode']
df = df.drop(outliers,axis=1)
df.head()

,Age,Sex,Caucasianeth,Smokerever,Smokerpresent,DM,Hypertension,Cerebrovasceventprevious,PAD,HFprevious,...,CKD_EPI,hsCRP,NTProBNP,NTProBNPgreater218,NTproBNP100,MCP1,Gal3,hsTroponinI,TWEAK,SBP
0,63,1,1,1,0,0,1,0,0,0,...,79.744541,1.588,37.3,0,0.373,151.60,9.86,0,123.48,150
1,51,0,1,1,0,1,1,0,0,0,...,65.387946,4.155,556,1,5.56,164.13,18.45,0,238.96,100
2,83,1,1,1,0,0,1,0,0,1,...,69.292452,4.194,1110,1,11.1,155.11,3.79,0.023,618.84,145
3,64,1,1,1,0,0,1,0,0,0,...,89.943755,0.738,35.9,0,0.359,106.86,8.66,0.003,130.13,140
4,81,1,1,1,0,1,1,0,0,0,...,56.371204,0.542,224,1,2.24,209.73,8.23,0.003,116.26,120


### Remove bad columns

In [8]:
def remove_bad_columns(df,bad_column_threshold):
    # find bad columns having too many missing values
    n_null = np.array(df.isnull().sum(axis=0))
    bad_col = np.array([]).astype(int)
    for i in range(len(n_null)):
        if n_null[i] >= bad_column_threshold:
            bad_col = np.append(bad_col,i)

    #print(bad_col)
    print('number of bad columns:',len(bad_col))

    # delete bad columns
    df = df.drop(df.columns[bad_col],axis=1)
    #df.info()
    return df  

In [9]:
df = remove_bad_columns(df,3)
df.info()

number of bad columns: 4
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 52 columns):
Age                               699 non-null int64
Sex                               699 non-null int64
Caucasianeth                      698 non-null object
Smokerever                        697 non-null object
Smokerpresent                     699 non-null int64
DM                                699 non-null int64
Hypertension                      699 non-null int64
Cerebrovasceventprevious          699 non-null int64
PAD                               699 non-null int64
HFprevious                        699 non-null int64
EFlower40                         699 non-null int64
AF                                699 non-null int64
TypelastACS                       699 non-null int64
PCIlastevent                      699 non-null int64
DESatlastevent                    699 non-null int64
CABGatlastevent                   699 non-null int64
ASA                 

### Remove bad rows

In [10]:
 # Find rows where target is missing
def find_missing_target_rows(df,target):
    # find rows where target is missing
    missing_row = df[target].isnull()
    print('Number of rows where target are missing:')
    print(sum(missing_row))

    #df = df[~missing_row]
    missing_row_indices = np.array([t for t in range(df.shape[0]) if missing_row[t]])
    
    return missing_row_indices

In [11]:
missing_target_rows = find_missing_target_rows(df,target)

Number of rows where target are missing:
0


We find bad rows which contain too many missing values, then remove them.

In [12]:
def find_bad_rows(df,bad_row_threshold):   
    # find bad rows having too many missing values
    n_null = np.array(df.isnull().sum(axis=1))
    bad_row = np.array([]).astype(int)
    for t in range(len(n_null)):
        if n_null[t] >= bad_row_threshold:
            bad_row = np.append(bad_row,t)

    #print(bad_row)
    print('number of bad rows:',len(bad_row))

    # delete bad rows
    #df = df.drop(bad_row)
    #df.info()
    return bad_row

In [13]:
bad_rows = find_bad_rows(df,3)
df.info()

number of bad rows: 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 52 columns):
Age                               699 non-null int64
Sex                               699 non-null int64
Caucasianeth                      698 non-null object
Smokerever                        697 non-null object
Smokerpresent                     699 non-null int64
DM                                699 non-null int64
Hypertension                      699 non-null int64
Cerebrovasceventprevious          699 non-null int64
PAD                               699 non-null int64
HFprevious                        699 non-null int64
EFlower40                         699 non-null int64
AF                                699 non-null int64
TypelastACS                       699 non-null int64
PCIlastevent                      699 non-null int64
DESatlastevent                    699 non-null int64
CABGatlastevent                   699 non-null int64
ASA                    

In [14]:
del_rows = np.union1d(missing_target_rows,bad_rows)
print('number of rows need to delete:',len(del_rows))

df = df.drop(del_rows)

number of rows need to delete: 1


### Separate target and attributes

In [15]:
dfx = df.drop(target,axis=1)
dfy = df[target]

### Find variable type

In [16]:
# number of uniques of each column (excluding NaN)
nu = np.array([len(pd.unique(dfx[col].dropna())) for col in dfx.columns])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[ 58   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2
   2   2   2   2   2   2   2   2   2   2   2   2  12 506 508   2 510   2
 508 124  59 132 218 134 388 656 516   2 516 626 503  68 669]


In [17]:
def define_variable_type(df,nu):
    i_binary = [] ; i_category = [] ; i_continuous = []
    for i in range(len(nu)):
        if nu[i] == 2: # binary 
            i_binary.append(i)
        elif nu[i] < 5: # !!!! NOTE: this is not always correct, depending on data
            i_category.append(i)
        else:
            i_continuous.append(i)

    print('i_binary:',i_binary)
    print('i_category:',i_category)   
    #i_binary, i_category, i_continuous
    
    variable_type  = np.ones(len(nu))  # binary
    variable_type[i_category] = 2   # categorical
    variable_type[i_continuous] = 3 # continuous

    return variable_type #,i_binary,i_category,i_continuous

In [18]:
variable_type = define_variable_type(dfx,nu)
print('variable type:',variable_type)

i_binary: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 33, 35, 45]
i_category: []
variable type: [3. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 3. 3. 3. 1. 3. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 1. 3. 3.
 3. 3. 3.]


### Impute missing values of attributes

In [19]:
def impute_missing(df,variable_type):
    # impute binary and categorical variables by the most frequency (in each column)
    # continuous variable by median
    # input: df: pandas data frame, variable_type: list
    # output: df2: pandas data frame
    df2 = df.copy()
    for i,col in enumerate(df.columns):
        if variable_type[i] < 3: # binary or caterogy
            df2[col] = df[col].fillna(df[col].mode().iloc[0])
        else: # continuous
            df2[col] = df[col].fillna(df[col].median())    
    return df2       

In [20]:
dfx_imputed = impute_missing(dfx,variable_type)
dfx_imputed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 698 entries, 0 to 697
Data columns (total 51 columns):
Age                               698 non-null int64
Sex                               698 non-null int64
Caucasianeth                      698 non-null object
Smokerever                        698 non-null object
Smokerpresent                     698 non-null int64
DM                                698 non-null int64
Hypertension                      698 non-null int64
Cerebrovasceventprevious          698 non-null int64
PAD                               698 non-null int64
HFprevious                        698 non-null int64
EFlower40                         698 non-null int64
AF                                698 non-null int64
TypelastACS                       698 non-null int64
PCIlastevent                      698 non-null int64
DESatlastevent                    698 non-null int64
CABGatlastevent                   698 non-null int64
ASA                               698 non-nul

## Data Processing

### Attributes

In [21]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))

        elif i_type == 2: # category
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))
            
        else: # continuous      
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))      

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [22]:
# convert x
x = np.array(dfx_imputed)
x_new = convert_binary_and_category(x,variable_type)

print(x_new.shape)
print(x_new)

(698, 51)
[[ 6.3000e+01  1.0000e+00  1.0000e+00 ...  9.8600e+00  0.0000e+00
   1.2348e+02]
 [ 5.1000e+01 -1.0000e+00  1.0000e+00 ...  1.8450e+01  0.0000e+00
   2.3896e+02]
 [ 8.3000e+01  1.0000e+00  1.0000e+00 ...  3.7900e+00  2.3000e-02
   6.1884e+02]
 ...
 [ 5.1000e+01  1.0000e+00  1.0000e+00 ...  8.9200e+00  0.0000e+00
   1.9940e+02]
 [ 4.7000e+01  1.0000e+00 -1.0000e+00 ...  1.8400e+00  0.0000e+00
   2.0718e+02]
 [ 5.9000e+01  1.0000e+00 -1.0000e+00 ...  5.4600e+00  5.0000e-03
   1.9600e+02]]


### Target

In [23]:
y = np.array(dfy)
#print(np.unique(y,return_counts=True))

# convert taget to 0 and 1
y_new = y
#y_new = np.ones(y.shape[0])
#y_new[y =='No'] = 0

print(np.unique(y_new,return_counts=True))

(array([0, 1]), array([675,  23]))


In [24]:
# combine X and y and save to a file
xy_new = np.hstack((x_new,y_new[:,np.newaxis]))
np.savetxt('data_processed.dat',xy_new,fmt='%f')